In [1]:
import os
import cv2
import numpy as np
import mediapipe as mp
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.utils import to_categorical


In [2]:
DATA_PATH = os.path.join('GYM_Data')

# Define the exercises and the number of sequences for each
exercises = ['curl_up', 'curl_down', 'squat_up', 'squat_down']
no_sequences = 30
sequence_length = 30

# Create directories
for exercise in exercises:
    for sequence in range(no_sequences):
        os.makedirs(os.path.join(DATA_PATH, exercise, str(sequence)), exist_ok=True)


In [3]:
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    return pose


In [7]:
DATA_PATH = os.path.join('GYM_Data')

# Define the exercises and the number of sequences for each
exercises = ['curl_up', 'curl_down', 'squat_up', 'squat_down']
no_sequences = 30
sequence_length = 30

In [8]:

# Prepare training data (X - features, y - labels)
X, y = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), f"{frame_num}.npy"))
            window.append(res)
        X.append(window)
        y.append(list(actions).index(action))

X = np.array(X)
y = to_categorical(y).astype(int)


In [9]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30, 33*4)))
model.add(LSTM(128, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])


In [10]:
model.fit(x_train, y_train, epochs=2000)  # Adjust epochs as needed


Epoch 1/2000


4/4 [==============================] - 5s 25ms/step - loss: 1.3553 - categorical_accuracy: 0.2632
Epoch 2/2000
4/4 [==============================] - 0s 25ms/step - loss: 1.1637 - categorical_accuracy: 0.5439
Epoch 3/2000
4/4 [==============================] - 0s 25ms/step - loss: 1.0635 - categorical_accuracy: 0.3860
Epoch 4/2000
4/4 [==============================] - 0s 25ms/step - loss: 0.9424 - categorical_accuracy: 0.6053
Epoch 5/2000
4/4 [==============================] - 0s 25ms/step - loss: 0.9082 - categorical_accuracy: 0.5526
Epoch 6/2000
4/4 [==============================] - 0s 25ms/step - loss: 0.8406 - categorical_accuracy: 0.6140
Epoch 7/2000
4/4 [==============================] - 0s 25ms/step - loss: 0.8216 - categorical_accuracy: 0.6228
Epoch 8/2000
4/4 [==============================] - 0s 24ms/step - loss: 0.7841 - categorical_accuracy: 0.6404
Epoch 9/2000
4/4 [==============================] - 0s 26ms/step - loss: 0.8520 - categorical_accuracy: 0.5526

In [11]:
model.save('gym.h5')

c:\Users\sanja\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
cap = cv2.VideoCapture(0)
counter = 0
stage = None

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        image, results = mediapipe_detection(frame, pose)
        keypoints = extract_keypoints(results)
        sequence = [keypoints]
        sequence = np.expand_dims(sequence, axis=0)
        
        predictions = model.predict(sequence)
        

        cv2.imshow('Mediapipe Feed', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()
